This notebook 1) aggregates and averages the five-fold results and 2) implements the model selection procedure based on the repeated cross-validation results. Final outputs are saved as "select_perf.csv."

In [ ]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#embed fonts
import matplotlib
matplotlib.rc('pdf', fonttype=42)
import scipy.stats
import glob
import json
from tqdm import tqdm

## name datasets

In [ ]:

dnames = [
    'HtnHeuri','HtnHypoKHeuri','ResHtnHeuri',
    'Htndx','HtnHypoKdx','ResHtndx',
]
dnames_nice = [
               'HTN Heuristic','HTN-Hypokalemia Heuristic',"Resistant HTN Heuristic",
               'HTN Diagnosis',"HTN-Hypokalemia Diagnosis","Resistant HTN Diagnosis",
]
dnames_to_nice = {k:v for k,v in zip(dnames, dnames_nice)}

## load all results

In [ ]:

rdir = 'resultsFinal_r1'
frames = []
feat_frames = []
#Read benchmark model results
for file in tqdm(glob.glob('../'+rdir+'/*/*/*.json')):
    print(file)
    with open(file,'r') as of:
        results = json.load(of)
    if 'Feat' in file:
        feat_frames.append(results)
    else:
        frames.append(results)


In [ ]:
df_results = pd.DataFrame.from_records(frames)
feat_df_results = pd.DataFrame.from_records(feat_frames)
for df in [df_results, feat_df_results]:
    print('results columns:',df.columns)    
    print('models:',df.model.unique())
    print('targets:',df.target.unique())

## manually add heuristic results

In [ ]:
from sklearn.metrics import average_precision_score, roc_auc_score
dx_to_heu = {k:v for k,v in zip(dnames[3:],dnames[:3])}
targets = {
            'htn_dx_ia':'Htndx',
            'res_htn_dx_ia':'ResHtndx', 
            'htn_hypok_dx_ia':'HtnHypoKdx', 
            'HTN_heuristic':'HtnHeuri', 
            'res_HTN_heuristic':'ResHtnHeuri',
            'hypoK_heuristic_v4':'HtnHypoKHeuri'
            }
rev_targets = {v:k for k,v in targets.items()}
drop_cols = ['UNI_ID'] + list(targets.keys())

print(dx_to_heu)
frames = []
for target in df_results.target.unique():
    if target in dx_to_heu.keys():
        heuristic = dx_to_heu[target]
        print('target:',target,'heuristic:',heuristic)
        target_raw = rev_targets[target]
        df_train = pd.read_csv('../Dataset' + str(101) + '/' + target + '/' + target + 'ATrain.csv')
        y_train = df_train[target_raw].values
        df_X_train = df_train.drop(drop_cols,axis=1)  
        df_test = pd.read_csv( '../Dataset' + str(101) + '/' + target + '/' + target + 'ATest.csv')
        df_X_test = df_test.drop(drop_cols,axis=1)  
        y_test = df_test[target_raw].values
        
        print(
            'phenotype cases:',np.sum(y_test==1)+np.sum(y_train==1),
            'phenotype controls:',np.sum(y_test==0)+np.sum(y_train==0),
              'out of',len(y_test)+len(y_train))
        y_heu = df_test[rev_targets[heuristic]]
        
        frames.append({'model':'Heuristic',
                       'target':target,
                       'RunID':101,
                       'average_precision_score_test': average_precision_score(y_test, y_heu),
                       'precision': np.sum((y_heu==1) & (y_test == 1))/ np.sum(y_heu==1),
                       'recall': np.sum((y_heu==1) & (y_test == 1))/ np.sum(y_test==1),
                       'specificity': np.sum((y_heu==0) & (y_test == 0))/ np.sum(y_test==0),
                       'roc_auc_score_test': roc_auc_score(y_test, y_heu)
                      })
df_heu = pd.DataFrame.from_records(frames)

In [ ]:
df_heu

In [ ]:
df_results = df_results.append(df_heu)

In [ ]:
df_results

In [ ]:
feat_df_results.info()

In [ ]:
for target, dfg in feat_df_results.groupby('target'):
    print('===============')
    print('target:',target)
    print('===============')
    for r in dfg['representation'].values:
        print(r)



## down-select FEAT models from runs using heuristic procedure

In [ ]:
from model_selection import select_feat_models, smallest_of_best_three_quartiles
import pandas as pd

feat_df_results_reduced = select_feat_models(feat_df_results, method= smallest_of_best_three_quartiles)


In [ ]:
print(feat_df_results_reduced)
# feat_df_results.target.unique()

In [ ]:
# combine dataframes
df_results = df_results.append(feat_df_results_reduced)


In [ ]:
df_results.isna().any()
df_results.model.unique()

In [ ]:
df_results.isna().any()

In [ ]:
df_results[df_results.model=='Feat_boolean_L1']

In [ ]:
# make nice labels
models = ['RandomForest',
          'DecisionTree',
          'Feat_boolean',
          'Feat_boolean_L1',
          'GaussianNaiveBayes',
          'LogisticRegressionCV_L2',
          'LogisticRegressionCV_L1',
          'Heuristic']
model_nice = ['RF',
          'DT',
          'FEAT',
          'FEAT L1',
          'GNB',
          'LR L2',
          'LR L1',
            'Heuristic'
             ]
nice_model_labels = {k:v for k,v in zip(models,model_nice)}
df_results['model_nice'] = df_results['model'].apply(lambda x: nice_model_labels[x])

### export df_results

In [ ]:
#Save output to select_perf.csv
import os
if not os.path.exists(rdir):
    os.mkdir(rdir)
df_results.to_csv(rdir + '/select_perf_final.csv')

In [ ]:
#Average values by run (average all folds)
df_results['size'] = df_results['size'].astype(float)
df_results_ave = df_results.groupby(['model_nice','target','RunID'], as_index=False).mean()
# df_results_ave = df_results.groupby(['model_nice','target','RunID','selection'], as_index=False).mean()
df_results_ave.head()

In [ ]:
df_results_ave.model_nice.unique()
df_results_ave[df_results_ave.model_nice=='FEAT'].target.unique()

## make docx table

In [ ]:
from docx import Document
from docx.shared import Inches, Pt

document = Document()

table = document.add_table(rows=1, cols=5)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Phenotype'
hdr_cells[1].text = 'Method'
hdr_cells[2].text = 'Test AUPRC'
hdr_cells[3].text = 'Test AUROC'
hdr_cells[4].text = 'Size'
# for target, dft in df_results_ave.groupby('target'):
i = 0
for target in dnames:
    print(target)
    dft = df_results_ave.loc[df_results_ave.target == target]
    j = 0
    models = ['GNB','DT','LR L1','LR L2','RF','FEAT']
    if 'dx' in target:
        models.append('Heuristic')
    for model in models:
        dftm = dft.loc[dft.model_nice == model]
        i += 1
        j += 1
        table.add_row()
        cells = table.rows[i].cells
        if j == 4:
            cells[0].text = dnames_to_nice[target]
        assert (len(dftm) == 1)
        cells[1].text = model
        cells[2].text = '{:0.2f}'.format( dftm['average_precision_score_test'].values[0])
        cells[3].text = '{:0.2f}'.format( dftm['roc_auc_score_test'].values[0])
        if model != 'Heuristic':
            cells[4].text = '{:d}'.format( int(dftm['size'].values[0]))
        else:
            cells[4].text = '-'
        print('-','\t\t'.join([c.text for c in cells[1:]]))
    print(50*'=') 
# for qty, id, desc in records:
#     row_cells = table.add_row().cells
#     row_cells[0].text = str(qty)
#     row_cells[1].text = id
#     row_cells[2].text = desc

document.add_page_break()

document.save('tables/Table_Final_Models.docx')